Module containing functions used with Med Associates output files

In [21]:
def get_file_info(file_path_name):
    # takes in a file name, finds the date, animal_number, and session and saves each accordingly
    import re
    
    path_split = file_path_name.split('/')
    file_split = path_split[-1].split('_')
    
    for row in file_split:
        if re.search(r'^......$', row):
            date = row
        if re.search(r'^...$', row):
            animal_number = row
        if re.search(r'^....$', row):
            session = row

    return date, animal_number, session

In [22]:
def make_array_from_path (file_path_name):
    # takes in path, creates array (each row of array is a row from original med file)
    import numpy as np
    import pandas as pd
    
    data = pd.read_table(file_path_name)
    data_table = pd.DataFrame(data = data)
    data_array = data_table.values
    
    return data_array

In [23]:
def get_eandt(array, event_letter, time_letter):
    #takes in array and the letters Med PC program uses to signify an event_stamp and time_stamp; 
    #finds all event_stamps and corresponding time_stamps; creats pandas dataframe
    import re
    import numpy as np
    import pandas as pd

    re_l = r"..[A-Z]\:..$"
    re_n = r'.. '

    i = 0
    j = 0
    k = 0
    event_data = []
    time_data = []
    for row in array:
        if re.search(re_l, str(row)):
            marker = str(row) 
            j = i
            k = i
            if re.match(marker, event_letter):
                j = j + 1
                while re.search(re_n, str(array[j])):
                    event_data.append((str(array[j])))
                    j = j + 1
            if re.match(marker, time_letter):
                k = k + 1
                while re.search(re_n, str(array[k])):
                    time_data.append((str(array[k])))
                    k = k + 1
            else:
                i = i + 1
        else:
            i = i + 1
            
    re_n = r'^\d(\d|\.)(\d|\.)(\d|\.)'
    event_data_list = []
    for row in event_data:
        data_set_split = row.split(" ")
        for line in data_set_split:
            if re.search(re_n, line):
                event_data_list.append(float(line[0:(len(line)-2)]))
                
    re_n = r'^\d(\d|\.)(\d|\.)(\d|\.)'
    time_data_list = []
    for row in time_data:
        data_set_split = re.split(" |'", row)
        for line in data_set_split:
            if re.search(re_n, line):
                time_data_list.append(float(line[0:(len(line)-1)]))

    eandt = np.column_stack((event_data_list, time_data_list))
    es_et = pd.DataFrame(data = eandt)
    es_et.columns = ['event_stamp', 'time_stamp']
    
    return es_et

In [24]:
def get_cs_and_iti(es_et, cs_s, cs_e, ITI_s, ITI_e):
    #takes in dataframe of events and corresponding time_stamps and the Med PC program code for cs and ITI start and end
    #finds time_stamps for each cs and ITI start and end, puts into new dataframe
    import numpy as np
    import pandas as pd
    
    cs_start = es_et[es_et.event_stamp == cs_s].reset_index()
    cs_end = es_et[es_et.event_stamp == cs_e].reset_index()
    ITI_start = es_et[es_et.event_stamp == ITI_s].reset_index()
    ITI_end = es_et[es_et.event_stamp == ITI_e].reset_index()
    
    cs_iti = pd.DataFrame(data = [cs_start.time_stamp, cs_end.time_stamp, ITI_start.time_stamp[:-1], ITI_end.time_stamp])
    cs_iti = cs_iti.T
    cs_iti.columns = ['CS_start', 'CS_end', 'ITI_start', 'ITI_end']
    
    return cs_iti

In [27]:
def PCA_analysis(cs_iti, es_et):
    #takes in dataframe of cs and ITI start and end times
    #takes in dataframe of all event_stamps and corresponding time_stamps
    #computes PCA values for each trial and session, places in new dataframe
    import numpy as np
    import pandas as pd
    
    PCA_values = pd.DataFrame()

    lever = -1
    lever_press = 0 #number of lever press/deflections during cs
    head_entries = 0 #number of head entries into the food cup during cs
    lever_prob = 0 #number of trials with a lever press, divided by the total number of trials
    head_prob = 0 #number of trials with a head entry, divided by the total number of trials
    ave_lat_lever = 0 #latency of first lever press averaged over the number of trials with a lever deflection
    ave_lat_head = 0 #latency of first head entry averaged over the number of trials with a head entry

    count_lever = 0 #running count of trials with a lever press
    count_head = 0 #running count of trials with a head entry
    lat_lever_sum = 0 #running sum of latency of first lever press for each trial with a lever press
    lat_head_sum = 0 #running sum of latency of first head entry for each trial with a head entry

    response_bias = 0 #ratio of lever presses and food cup entries in relation to total number of responses
    prob_diff = 0 #the difference between the probability of pressing the lever and the probability of entering the food cup
    lat_score = 0 #difference between the latencies to approach the lever and the food cup
    PCA_score = 0 #ave of above three scores (-1 is completely goal-tracking; +1 is completely sign-tracking)

    i = 0

    while i < 25:
        data = es_et[(es_et.time_stamp >= cs_iti.CS_start[i]) & (es_et.time_stamp <= cs_iti.CS_end[i])]

        if len(data[data.event_stamp == 1]) > 0:
            lever = 1
        if len(data[data.event_stamp == 2]) > 0:
            lever = 2
            
        if lever > 0:
            lever_press = lever_press + len(data[data.event_stamp == lever])
    
        head_entries = head_entries + len(data[data.event_stamp == 6])

        if len(data[data.event_stamp == lever]) > 0:
            count_lever = count_lever + 1
            lat_lever = data[data.event_stamp == lever].time_stamp.iloc[0] - cs_iti.CS_start[i]
            lat_lever_sum = lat_lever_sum + lat_lever
        
        if len(data[data.event_stamp == 6]) > 0:
            count_head = count_head + 1
            lat_head = data[data.event_stamp == 6].time_stamp.iloc[0] - cs_iti.CS_start[i]
            lat_head_sum = lat_head_sum + lat_head
        
        i = i + 1
    
    lever_prob = count_lever / 25
    head_prob = count_head / 25

    if lat_lever_sum > 0:
        ave_lat_lever = lat_lever_sum / count_lever
    if lat_head_sum > 0:
        ave_lat_head = lat_head_sum / count_head
    
    if (lever_press > 0) | (head_entries > 0):
        response_bias = (lever_press - head_entries) / (lever_press + head_entries)

    prob_diff = lever_prob - head_prob
    lat_score = (ave_lat_head - ave_lat_lever) / 10

    PCA_score = (response_bias + prob_diff + lat_score) / 3

    PCA_values['lever_press'] = [lever_press]
    PCA_values['head_entries'] = head_entries
    PCA_values['lever_prob'] = lever_prob
    PCA_values['head_prob'] = head_prob
    PCA_values['ave_lat_lever'] = ave_lat_lever
    PCA_values['ave_lat_head'] = ave_lat_head
    PCA_values['response_bias'] = response_bias
    PCA_values['prob_diff'] = prob_diff
    PCA_values['lat_score'] = lat_score
    PCA_values['PCA_score'] = PCA_score

    return PCA_values